In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from functools import reduce
import tabula
%matplotlib inline

* participants in Wave IV are 24 - 32 years old
* Census data in thousands

Census

In [ ]:
census_all = pd.read_excel(r"C:\Users\mnorm\Documents\NSS\NSS-Capstone\data\National Longitudinal Study of Adolescent to Adult Health\Census Educational Attainment - 2008.xls", 'all')
census_all.head()

In [ ]:
censusall_sub = census_all.iloc[3:5]
censusall_sub

In [ ]:
censusall_sub.loc['Total'] = censusall_sub.iloc[:, :].sum()

In [ ]:
censusall_sub.reset_index(drop = False)

In [ ]:
census_pct = censusall_sub.iloc[1:, 1:].div(censusall_sub['Total'], axis = 0)

In [ ]:
for col in censusall_sub.columns[2:] :   
    census_pct[col] = (censusall_sub[col]/censusall_sub['Total'])

In [ ]:
census_pct = round(census_pct, 4)
census_pct

Merge Wave AIDs

In [ ]:
wave1 = pd.read_csv(r"C:\Users\mnorm\Documents\NSS\NSS-Capstone\data\National Longitudinal Study of Adolescent to Adult Health\ICPSR_21600\DS0001\21600-0001-Data.tsv", sep = '\t', usecols = ['AID'])
wave3educ = pd.read_csv(r"C:\Users\mnorm\Documents\NSS\NSS-Capstone\data\National Longitudinal Study of Adolescent to Adult Health\ICPSR_21600\DS0016\21600-0016-Data.tsv", sep = '\t', usecols = ['AID'])
wave3grad = pd.read_csv(r"C:\Users\mnorm\Documents\NSS\NSS-Capstone\data\National Longitudinal Study of Adolescent to Adult Health\ICPSR_21600\DS0017\21600-0017-Data.tsv", sep = '\t', usecols = ['AID'])
wave4 = pd.read_csv(r"C:\Users\mnorm\Documents\NSS\NSS-Capstone\data\National Longitudinal Study of Adolescent to Adult Health\ICPSR_21600\DS0022\21600-0022-Data.tsv", sep = '\t', usecols = ['AID'])

In [ ]:
print('Count of Responses per Wave')
print('Wave 1: ', len(wave1))
print('Wave 3 education: ', len(wave3educ))
print('Wave 3 graduation: ', len(wave3grad))
print('Wave 4: ', len(wave4))

In [ ]:
waves = [wave1, wave3educ, wave3grad, wave4]
AID = reduce(lambda left, right: pd.merge(left, right, on = 'AID', how = 'inner'), waves)
len(AID)

Highest Education Level

In [ ]:
highest_educ = pd.read_excel(r"C:\Users\mnorm\Documents\NSS\NSS-Capstone\data\Wave 4 Questionaire .xlsx", 'highest_educ_level')
highest_educ = highest_educ.rename(columns = highest_educ.iloc[0])
highest_educ = highest_educ.drop(highest_educ.index[0])
highest_educ = highest_educ.drop(['Value'], axis = 1)
highest_educ

In [ ]:
census_pct

In [ ]:
most_recent_degree = pd.read_excel(r"C:\Users\mnorm\Documents\NSS\NSS-Capstone\data\Wave 4 Questionaire .xlsx", 'most_recent_degree')
most_recent_degree = most_recent_degree.rename(columns = most_recent_degree.iloc[0])
most_recent_degree = most_recent_degree.drop(most_recent_degree.index[0])
#most_recent_degree = most_recent_degree.drop(['Value'], axis =1)
most_recent_degree
#legitimate skip represents those who did not complete high school

Did students who had greater academic success in hs (gpa above 3), attain a higher level of education?

In [ ]:
hs_gpa = pd.read_csv(r"C:\Users\mnorm\Documents\NSS\NSS-Capstone\data\National Longitudinal Study of Adolescent to Adult Health\ICPSR_21600\DS0016\21600-0016-Data.tsv", sep = '\t', usecols = ['AID', 'EAOGPAC'])
hs_gpa = hs_gpa.rename(columns = {'AID': 'AID', 'EAOGPAC': 'cumulative_gpa'})
hs_gpa = pd.merge(AID, hs_gpa, on = 'AID', how = 'left')
#hs_gpa.isna().sum() #check for nulls to make sure the AID matched
hs_gpa

In [ ]:
hs_gpa.dtypes

In [ ]:
#remove students with 9995 value as cumulative gpa. those students didn't receive grades
#len(hs_gpa.loc[hs_gpa['cumulative_gpa'] == 9995]) =24 students
hs_gpa = hs_gpa.drop(hs_gpa[hs_gpa['cumulative_gpa'] == 9995].index)

In [ ]:
educ_attainment = pd.read_csv(r"C:\Users\mnorm\Documents\NSS\NSS-Capstone\data\National Longitudinal Study of Adolescent to Adult Health\ICPSR_21600\DS0022\21600-0022-Data.tsv", sep = '\t', usecols = ['AID', 'H4ED3A'])
educ_attainment = educ_attainment.rename(columns = {'H4ED3A':'Value'})
educ_attainment.dtypes

In [ ]:
mrd_code = most_recent_degree[['Value', 'Description']]
mrd_code.dtypes

In [ ]:
mrd_code['Value'] = mrd_code['Value'].astype('int')

In [ ]:
educ_attainment = pd.merge(educ_attainment, mrd_code, on = 'Value', how = 'left')

In [ ]:
gpa_educ = pd.merge(hs_gpa, educ_attainment, on = 'AID', how = 'left')
#gpa_educ.isna().sum()
bins = [-1, 1, 2, 3 ,4]
labels = ['0-1', '1-2', '2-3', '3-4']
gpa_educ['gpa_range'] = pd.cut(np.array(gpa_educ['cumulative_gpa']), bins = bins, labels = labels)
pd.DataFrame(gpa_educ.groupby('Description')['cumulative_gpa'].min())

In [ ]:
pd.DataFrame(gpa_educ.groupby('Description')['cumulative_gpa'].max())

In [ ]:
pd.DataFrame(gpa_educ.groupby('Description')['cumulative_gpa'].mean())

Did students who had greater academic success in hs(gpa above 3), express a desire for more education?

In [ ]:
hs_gpa

In [ ]:
desired_educ = pd.read_csv(r"C:\Users\mnorm\Documents\NSS\NSS-Capstone\data\National Longitudinal Study of Adolescent to Adult Health\ICPSR_21600\DS0022\21600-0022-Data.tsv", sep = '\t', usecols = ['AID','H4ED8'])
desired_educ = desired_educ.rename(columns = {'H4ED8': 'Value'})
expected_educ = pd.read_csv(r"C:\Users\mnorm\Documents\NSS\NSS-Capstone\data\National Longitudinal Study of Adolescent to Adult Health\ICPSR_21600\DS0022\21600-0022-Data.tsv", sep = '\t', usecols = ['AID','H4ED9'])

In [ ]:
desired_educ_level = pd.read_excel(r"C:\Users\mnorm\Documents\NSS\NSS-Capstone\data\Wave 4 Questionaire .xlsx", 'desired_educ_level')
desired_educ_level = desired_educ_level.rename(columns = desired_educ_level.iloc[0])
desired_educ_level = desired_educ_level.drop(desired_educ_level.index[0])
desired_educ_level = desired_educ_level[['Value', 'Description']]
desired_educ_level

In [ ]:
desired_educ_level['Value'] = desired_educ_level['Value'].astype(int)
desired_educ_level.dtypes

In [ ]:
desired_educ = pd.merge(desired_educ, desired_educ_level, on = 'Value', how = 'left' )
desired_educ

In [ ]:
gpa_desired_educ = pd.merge(hs_gpa, desired_educ, how = 'left', on = 'AID' )
#gpa_desired_educ.isna().sum()
gpa_desired_educ

In [ ]:
gpa_desired_educ.groupby('Description')['cumulative_gpa'].min()

In [ ]:
gpa_desired_educ.groupby('Description')['cumulative_gpa'].max()

In [ ]:
gpa_desired_educ.groupby('Description')['cumulative_gpa'].mean()

demographics of people who want higher education

demographics of people who want higher education but feel like they can't get it

close to peers and grades

Was there a year where gpa dropped among students?